<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/10_adet_csv_olu%C5%9Fturma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
import requests

# Kullanıcının seçebileceği coinler
available_coins = ["bitcoin", "ethereum", "cardano", "binancecoin", "solana", "ripple", "dogecoin", "polkadot", "litecoin", "chainlink"]

# Kullanıcının seçtiği coinleri belirleme
selected_coins = available_coins[:10]  # İlk 10 coini seçiyoruz

# Boş DataFrame oluşturma
all_data = pd.DataFrame()

for coin in selected_coins:
    url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
    params = {
        "vs_currency": "usd",
        "days": "1",
        "interval": "daily"
    }

    response = requests.get(url, params=params)
    data_json = response.json()

    if "prices" in data_json:
        prices = data_json["prices"]
        df = pd.DataFrame(prices, columns=["timestamp", "close"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        df["coin"] = coin
        all_data = pd.concat([all_data, df], ignore_index=True)
    else:
        print(f"{coin} için veri alınamadı.")

# CSV'ye kaydetme
all_data.to_csv("multi_coin_price_data.csv", index=False)
print("Veri başarıyla indirildi ve 'multi_coin_price_data.csv' olarak kaydedildi.")

# CSV'den veri yükleme
data = pd.read_csv("multi_coin_price_data.csv")
data.dropna(inplace=True)

# Hareketli ortalamalar ve göstergeleri hesaplama
ema_length = 20
rsi_length = 14
bollinger_length = 20
bollinger_std_dev = 2.0
rsi_ema_length = 200
bollinger_ema_length = 200

# Her coin için göstergeleri hesaplama
data["EMA"] = data.groupby("coin")["close"].transform(lambda x: x.ewm(span=ema_length, adjust=False).mean())

delta = data.groupby("coin")["close"].diff(1)
gain = delta.where(delta > 0, 0).groupby(data["coin"]).transform(lambda x: x.rolling(window=rsi_length).mean())
loss = -delta.where(delta < 0, 0).groupby(data["coin"]).transform(lambda x: x.rolling(window=rsi_length).mean())
rs = gain / loss
data["RSI"] = 100 - (100 / (1 + rs))

data["MiddleBand"] = data.groupby("coin")["close"].transform(lambda x: x.rolling(window=bollinger_length).mean())
data["StdDev"] = data.groupby("coin")["close"].transform(lambda x: x.rolling(window=bollinger_length).std())
data["UpperBand"] = data["MiddleBand"] + (bollinger_std_dev * data["StdDev"])
data["LowerBand"] = data["MiddleBand"] - (bollinger_std_dev * data["StdDev"])

data["RSI_EMA"] = data.groupby("coin")["RSI"].transform(lambda x: x.ewm(span=rsi_ema_length, adjust=False).mean())
data["Bollinger_EMA"] = data.groupby("coin")["MiddleBand"].transform(lambda x: x.ewm(span=bollinger_ema_length, adjust=False).mean())

# Alım/Satım Sinyalleri
buy_signal = (data["close"] > data["EMA"]) & (data["RSI"] < 30) & (data["close"] < data["LowerBand"])
sell_signal = (data["close"] < data["EMA"]) & (data["RSI"] > 70) & (data["close"] > data["UpperBand"])

data["BuySignal"] = buy_signal
data["SellSignal"] = sell_signal

data["NashEquilibrium"] = ~(buy_signal & sell_signal)

data["CombinedProbability"] = ((data["close"] - data["EMA"]) / data["close"] + (data["RSI"] - 50) / 50 + (data["close"] - data["MiddleBand"]) / (data["UpperBand"] - data["LowerBand"])) / 3
data["LogisticProbability"] = 1 / (1 + np.exp(-data["CombinedProbability"]))

data["FinalBuySignal"] = data["BuySignal"] & (data["LogisticProbability"] > 0.5) & data["NashEquilibrium"]
data["FinalSellSignal"] = data["SellSignal"] & (data["LogisticProbability"] < 0.5) & data["NashEquilibrium"]

# CSV ve Excel çıktısı oluşturma
data.to_csv("multi_coin_processed_signals.csv", index=False)
data.to_excel("multi_coin_processed_signals.xlsx", index=False)
print("Veri başarıyla 'multi_coin_processed_signals.csv' ve 'multi_coin_processed_signals.xlsx' olarak kaydedildi.")

print(data[["coin", "close", "FinalBuySignal", "FinalSellSignal"]].tail())


ripple için veri alınamadı.
dogecoin için veri alınamadı.
polkadot için veri alınamadı.
litecoin için veri alınamadı.
chainlink için veri alınamadı.
Veri başarıyla indirildi ve 'multi_coin_price_data.csv' olarak kaydedildi.
Veri başarıyla 'multi_coin_processed_signals.csv' ve 'multi_coin_processed_signals.xlsx' olarak kaydedildi.
          coin       close  FinalBuySignal  FinalSellSignal
5      cardano    0.701726           False            False
6  binancecoin  614.709487           False            False
7  binancecoin  614.491397           False            False
8       solana  199.709394           False            False
9       solana  204.655905           False            False
